In [1]:
%%capture
!sudo curl -L https://github.com/dolthub/dolt/releases/latest/download/install.sh | sudo bash
!pip install pymysql
!dolt clone onefact/paylesshealth
  

In [2]:
!cd paylesshealth/ && dolt sql -q "select * from hospitals where cdm_url is not null;" -r csv > hospitals.csv

In [24]:
import pandas as pd
# df = pd.read_csv('paylesshealth/hospitals.csv')
df = pd.read_csv('non-null.csv')
df = df.head(50)

In [2]:
import aiohttp
import asyncio
import re
import cgi
from pathlib import Path
from urllib.parse import urlparse

In [3]:
def is_url(url):
    try:
        a = urlparse(url)
        return all([a.scheme, a.netloc])
    except ValueError:
        return False

In [28]:
async def checker(session, url):
    if pd.isna(url):
        return pd.NA

    
    headers= {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    "Host": urlparse(url).netloc,
    "Referer": urlparse(url).netloc
    }
    try:
        async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, headers=headers) as r:
            if r.status == 200:
                return url
            elif r.status == 404:
                return pd.NA
            elif r.status == 301 or r.status == 302:
                new_loc = r.headers.get('location')
                if is_url(new_loc):
                    return new_loc
                else:
                    return pd.NA
            elif r.status != 200 and r.status != 404:
#                 print(f"\nUnkknown code: {r.status} {url}")
                return url

    except aiohttp.ClientResponseError as e:
        print("ERROR", e, url)
        return pd.NA

In [13]:
from tqdm.asyncio import tqdm

async def main():
    async with aiohttp.ClientSession(raise_for_status = True) as session:
        cols = ["homepage", "cdm_url", "cdm_indirect_url"]
        for col in cols:
            df[col] = await tqdm.gather(*(checker(session, url) for url in df[col]))
        return df

In [ ]:
t = await main()







  0%|                                                                                           | 0/50 [00:00<?, ?it/s]





  2%|█▋                                                                                 | 1/50 [00:00<00:17,  2.86it/s]





 10%|████████▎                                                                          | 5/50 [00:00<00:04,  9.31it/s]





 24%|███████████████████▋                                                              | 12/50 [00:00<00:01, 22.64it/s]





 38%|███████████████████████████████▏                                                  | 19/50 [00:00<00:00, 31.56it/s]





 48%|███████████████████████████████████████▎                                          | 24/50 [00:00<00:00, 32.77it/s]





 58%|███████████████████████████████████████████████▌                                  | 29/50 [00:01<00:00, 33.51it/s]





 66%|██████████████████████████████████████████████████████                            | 33/50 [00:01<00:00, 27.92it/s]

ERROR 0, message='', url=URL('https://www.gch.org/') https://www.gch.org/








 80%|█████████████████████████████████████████████████████████████████▌                | 40/50 [00:01<00:00, 15.96it/s]





 86%|██████████████████████████████████████████████████████████████████████▌           | 43/50 [00:02<00:00, 17.20it/s]

ERROR 400, message='Got more than 8190 bytes (12517) when reading Header value is too long.', url=URL('https://www.dmc.org/') https://www.dmc.org/








 92%|███████████████████████████████████████████████████████████████████████████▍      | 46/50 [00:02<00:00, 14.77it/s]





 96%|██████████████████████████████████████████████████████████████████████████████▋   | 48/50 [00:02<00:00, 15.49it/s]

In [13]:
t

,ccn,name,state,city,homepage,cdm_indirect_url,cdm_url
0,050054,SAN GORGONIO MEMORIAL HOSPITAL,CA,BANNING,https://sgmh.org/,https://sgmh.org/,https://www.cdmpricing.com/8488f242c158e777c19...
1,140062,PALOS COMMUNITY HOSPITAL,IL,PALOS HEIGHTS,https://www.nm.org/,https://www.nm.org/patients-and-visitors/billi...,https://www.nm.org/-/media/northwestern/resour...
2,050055,CALIFORNIA PACIFIC MEDICAL CENTER - MISSION BE...,CA,SAN FRANCISCO,https://www.sutterhealth.org/,https://www.sutterhealth.org/for-patients/heal...,https://www.sutterhealth.org/pdf/chargemaster/...
3,140063,RUSH OAK PARK HOSPITAL,IL,OAK PARK,https://www.rush.edu/,https://www.rush.edu/patients-visitors/billing...,https://www.allegiancecosttransparency.com/202...
4,050056,ANTELOPE VALLEY HOSPITAL,CA,LANCASTER,https://www.avmc.org/,https://www.avmc.org/patients/billing-insuranc...,https://www.cdmpricing.com/f961a5b495c6b41f2fc...
5,050057,KAWEAH HEALTH MEDICAL CENTER,CA,VISALIA,https://www.kaweahhealth.org/,https://www.kaweahhealth.org/patients-visitors...,https://www.kaweahhealth.org/images/94-1534475...
6,140064,ST MARY MEDICAL CENTER,IL,GALESBURG,https://www.osfhealthcare.org/,https://www.osfhealthcare.org/osf/forms/price-...,https://www.osfhealthcare.org/media/filer_publ...
7,140065,ADVENTHEALTH LA GRANGE,IL,LA GRANGE,https://www.amitahealth.org/,https://www.adventhealth.com/pricelist,https://www.adventhealth.com/sites/default/fil...
8,140067,SAINT FRANCIS MEDICAL CENTER,IL,PEORIA,https://www2.osfhealthcare.org/,https://www.osfhealthcare.org/osf/forms/price-...,https://www.osfhealthcare.org/media/filer_publ...
9,140068,ROSELAND COMMUNITY HOSPITAL,IL,CHICAGO,https://www.roselandhospitaltalks.org/,https://www.roselandhospitaltalks.org/cost-of-...,https://apps.para-hcfs.com/PTT/FinalLinks/Repo...
